In [1]:
from src.data._dataobject import DataObject
from src.data.types._text import String, NSString
from src.data.types._nested import Nested, NSNested
# from src.data.types._list import List
from src.data.types._int import Int
from graphene import ObjectType, Schema, Field, List

In [2]:
class City(DataObject):
    name = NSString()
    country = NSString(max_length=3, min_length=2, uppercase=True)



In [3]:
City.graphql()

<GQLABCMeta meta=<ObjectTypeOptions name='GQLABCMeta'>>

In [4]:
class Query(ObjectType):
    cities = List(City.get_graphql_type(City()))

    def resolve_cities(root, info, *args, **kwargs):
        return [
            City(name='Piacenza', country='it').get_graphql_type(),
            City(name='Milan', country='it').get_graphql_type(),
            City(name='Kigali', country='rw')
        ]

In [5]:
schema = Schema(query=Query)

In [6]:
res = schema.execute('query {  cities { name country } }')

In [7]:
m = [City(**x) for x in res.data['cities']]

In [8]:
City.filter_re_rule(m, 'country=="IT"')

[<City>, <City>]

In [9]:
from cryptography.fernet import Fernet

In [10]:
Fernet.generate_key()

b'pYmUTQG8OWH2vyMrm5EAIw7PLXXXspZBz0YfW2Y9HAg='

In [11]:
fr = Fernet(Fernet.generate_key())

In [12]:
enc = fr.encrypt(bytes('hello', 'utf-8'))

In [13]:
fr.decrypt(enc).decode('utf-8')

'hello'

In [14]:
enc.decode('utf-8')

'gAAAAABh7ymOLQfMQAjfk8UNtyILPTAwlx7Vx6eYw_ty7sbnLjX4CllDvoU3c1Atc411K_0itAik43A_QOdclSaE9DFMFHAtuA=='

In [15]:
from src.data.types._password import Password
from src.data.types._text import NSString

class User(DataObject):
    username = NSString()
    password = Password(Fernet.generate_key())

u = User(username='FrodoBaggins', password='tequila!')

In [16]:
u.password == 'tequila!'

False

In [17]:
u.password

'gAAAAABh7ymOw-NZgy1rH22Xd1_GiThwalhsaoWztVKlU2ylNOAcDOQTNa_458Jj83A_VgcPeRDdFn9UUZDcaYTBKDvZNtgipQ=='

In [112]:
from typing import Union

class EncryptedString:
    _key: bytes = NotImplemented
    _val: str
    _f = False

    @property
    def value(self) -> str:
        return Fernet(self._key).decrypt(self._val.encode('utf-8')).decode('utf-8')

    @value.setter
    def value(self, val: str) -> None:
        self._val = Fernet(self._key).encrypt(val.encode('utf-8')).decode('utf-8')

    @property
    def raw(self) -> str:
        return self._val

    @raw.setter
    def raw(self, val: str) -> None:
        self._val = val

    def __eq__(self, __o: object) -> bool:
        if isinstance(__o, str):
            __o = Fernet(self._key).encrypt(
                __o.encode('utf-8')).decode('utf-8')
        elif isinstance(__o, bytes):
            __o = Fernet(self._key).encrypt(
                __o).decode('utf-8')
        return super().__eq__(__o)

    def __eq__(self, __o: object) -> bool:
        return __o == self.value or __o == self.raw

    def __init__(self, val: str, key: Union[str, bytes, None] = None) -> None:
        if key is None:
            key = Fernet.generate_key()
        if isinstance(key, str):
            key = bytes(key, 'utf-8')
        self._key = key
        self.value = val

    @classmethod
    def from_raw(cls, raw, key):
        c = cls('', key)
        c.raw = raw
        return c

    def __str__(self) -> str:
        return self.raw

In [113]:
g = EncryptedString('tequila!')

In [114]:
g == 'tequila!'

True

In [115]:
g.value

'tequila!'

In [116]:
g.value = 'tequila?'

In [117]:
g.value

'tequila?'

In [118]:
g.raw

'gAAAAABh7zL92XjPQH4bbvSFNBEWbBDmkSuUZ-34Iv42CqEjlYNA9uXQ3LH0R_PG9FHdKHimsZ4DxX3fb4QfxN8YBywskczmIA=='

In [119]:
g2 = EncryptedString.from_raw(g.raw, g._key)

In [120]:
g2.value

'tequila?'

In [121]:
g2 == g.raw

True

In [122]:
g.value = 'tequila!'

In [123]:
g2 == g.raw

False

In [1]:
str(g)

NameError: name 'g' is not defined

In [2]:
from pendulum.datetime import DateTime

DateTime.now()

DateTime(2022, 1, 25, 9, 32, 51, 398823, tzinfo=Timezone('Europe/Berlin'))

In [3]:
from datetime import datetime

In [4]:
now = DateTime.now()

In [5]:
datetime(now)

TypeError: 'DateTime' object cannot be interpreted as an integer

In [7]:
from pendulum import parse

In [12]:
parse('21/01/2022', strict=False)

DateTime(2022, 1, 21, 0, 0, 0, tzinfo=Timezone('UTC'))

In [13]:
now

DateTime(2022, 1, 25, 9, 33, 23, 506410, tzinfo=Timezone('Europe/Berlin'))

In [14]:
now.to_datetime_string()

'2022-01-25 09:33:23'

In [15]:
now.tzinfo

Timezone('Europe/Berlin')

In [16]:
datetime.now().tzinfo

In [17]:
dtn = datetime.now()


In [18]:
dtn.tzinfo = now.tzinfo

AttributeError: attribute 'tzinfo' of 'datetime.datetime' objects is not writable

In [19]:
datetime.now(now.tzinfo)

datetime.datetime(2022, 1, 25, 9, 53, 23, 926311, tzinfo=Timezone('Europe/Berlin'))

In [21]:
now.to_datetime_string()


'2022-01-25 09:33:23'

In [22]:
datetime.strptime(now.to_datetime_string())

TypeError: strptime() takes exactly 2 arguments (1 given)

In [23]:
isinstance(now, datetime)

True

In [26]:
from pendulum import parse

parse('now', strict=True)

DateTime(2022, 1, 25, 17, 8, 20, 708310, tzinfo=Timezone('Europe/Berlin'))

In [28]:
parse('today', strict=False)

ParserError: Invalid date string: today

In [29]:
val = 'now'

In [30]:
isinstance(_new_val := parse(val, strict=False), datetime)

True

In [31]:
_new_val

DateTime(2022, 1, 25, 18, 8, 7, 589709, tzinfo=Timezone('Europe/Berlin'))